In [ ]:
import os
from subprocess import call
import pickle as pk
from IPython.display import clear_output
current_dir = os.listdir('./')
for path in os.listdir('../25 3 2019/'):
    if os.path.islink(path) is not True and path not in current_dir:
        call(['ln','-s','../25 3 2019/'+ path, path])        
from cfg import *
os.chdir('../CCP_Collab_Backup_2/')
import PrimaryLibrary as PL
imp.reload(PL)
import MorphologyLibrary as MLab
imp.reload(MLab)
import Allen_API_Library as API
imp.reload(API)
MesoPred = PL.MesoconnectomePredictor()

### Update the MesoPred params with information regarding subcortical and thalamic indices

In [ ]:
from allensdk.api.queries.ontologies_api import OntologiesApi

oapi = OntologiesApi()
mcc = MouseConnectivityCache()
structure_tree = mcc.get_structure_tree()

# get the ids of all the structure sets in the tree
structure_set_ids = structure_tree.get_structure_sets()

# query the API for information on those structure sets
pd.DataFrame(oapi.get_structure_sets(structure_set_ids))

summary_structures = structure_tree.get_structures_by_set_id([687527670])
subcortical_structures = []
thalamic_structures    = []
cortical_structures    = []
for val in summary_structures:
    if val['acronym'] == 'TH':
        kids = structure_tree.descendant_ids([val['id']])
        thalamic_structures.extend(kids[0])
    if val['acronym'] == 'Isocortex': 
        ids = structure_tree.descendant_ids([val['id']])
        cortical_structures.extend(kids[0])
    else:
        kids = structure_tree.descendant_ids([val['id']])
        subcortical_structures.extend(kids[0])
     
with open('structures.csv','rb') as fp:
    structure       = PL.readtable(fp) 
    struct_idx_dict = {val:idx for idx,val in enumerate(structure['id'])}
for idx,val in enumerate(thalamic_structures):
    thalamic_structures[idx] = struct_idx_dict[str(val)]
for idx,val in enumerate(subcortical_structures):    
    subcortical_structures[idx] = struct_idx_dict[str(val)] 
for idx,val in enumerate(cortical_structures):    
    cortical_structures[idx] = struct_idx_dict[str(val)] 
    
MesoPred.targetprofiles['cortical structures'] = cortical_structures   
cre_dict_2 = OrderedDict([(key,value['layer'][0] + ' ' + value['cell-type'][0])  for key,value in MesoPred.ConDict.items() if 'layer' in value.keys()])
cre_dict_2 = pd.DataFrame(cre_dict_2, index = ['cre-profiles']).T
cre_dict_2.to_csv('paper1_figures/cre_profiles.csv')

In [ ]:
GeneExp        = h5py.File('G_Exp.hdf5', 'r')['dataset1']
a,b,c,d  = MesoPred.PreProcessing(GeneExp[MesoPred.params['leaf_keys'],:], 
                                  MesoPred.params,
                                  MesoPred.ConDict['wild_type']['ConMat'][MesoPred.params['leaf_keys'],:])
ClfResults_ridge = {}; ridge_pred = {}
MesoPred.params           = pk.load(open('updated_params.pkl','rb'))
for key in MesoPred.ConDict.keys():
    if os.path.isfile(MesoPred.params['prefix'] + key + '_all_results' + '_ridge.pkl') == True:
        infile = MesoPred.params['prefix'] + key + '_all_results' + '_ridge.pkl'
        print infile
        tmp = pk.load(open(infile,'r'))
        ClfResults_ridge[key] = {key:val for key,val in tmp.items()}
         
clear_output()

In [ ]:
measure  = 'AURoc'
MetaInfo = {}
for key in ClfResults_ridge.keys():
    tmp1 = [val[0] for val in ClfResults_ridge[key][measure]] 
    sort1 = np.argsort(tmp1)[::-1]; sort3 = np.sort(tmp1)[::-1]
    set_length = len(sort1)   

    MetaInfo['min1']  = [MesoPred.ConDict[key]['structure-abbrev'][sort1[set_length-1]], sort3[set_length-1]]
    MetaInfo['max1']  = [MesoPred.ConDict[key]['structure-abbrev'][sort1[0]], sort3[0]]
    MetaInfo['med1']  = [MesoPred.ConDict[key]['structure-abbrev'][sort1[set_length/2]], sort3[set_length/2]]

    MetaInfo['save_file']  = key + '_' + measure +'.jpg'
    MetaInfo['save_file2'] = key + '_' + measure +'.eps'
    MetaInfo['title']      = key  
    MetaInfo['ylabel']     = measure
    MetaInfo['xlabel']     = 'Ridge'
    MetaInfo['lb']         = np.min(ClfResults_ridge[key][measure]) - 0.1
    MetaInfo['ub']         = 1.0
   
    print key, np.median(ClfResults_ridge[key][measure]), sci.stats.iqr(ClfResults_ridge[key][measure])
    fsz = 14
    plt.rcParams['axes.facecolor'] = 'white'
    plt.figure(figsize=(10,6))
    plt.tight_layout()
    plt.rcParams['figure.figsize']
    box = plt.boxplot(ClfResults_ridge[key][measure], 0, 'gD', widths = 0.6, whis = [5,95])
    ax = plt.gca()
    ax.xaxis.set_tick_params(labelsize = 16)
    ax.yaxis.set_tick_params(labelsize = 16)
    plt.text(0.55, MetaInfo['max1'][1], MetaInfo['max1'][0], fontsize = fsz)
    plt.text(0.55, MetaInfo['min1'][1], MetaInfo['min1'][0], fontsize = fsz)
    plt.text(0.55, MetaInfo['med1'][1], MetaInfo['med1'][0], fontsize = fsz)
    plt.title(MetaInfo['title'], fontsize = 20)
    plt.xlabel(MetaInfo['xlabel'], fontsize = 18)
    plt.ylabel(MetaInfo['ylabel'], fontsize = 18)
    
    for key2 in box.keys():
        for bp in box[key2]:
            bp.set(linewidth = 2)
            
    if key == 'wild_type':
        lb = 0.7
    else:
        lb = 0.8
    plt.yticks(np.arange(0.7,1.01,0.05))
    #plt.yticks(np.arange(0.4,1.0,0.05))

    plt.savefig('paper1_figures/regression_accuracy_figures/' + MetaInfo['save_file'])
    plt.savefig('paper1_figures/regression_accuracy_figures/' + MetaInfo['save_file2'])
    plt.show(block = False)
    plt.pause(1)
    plt.close()
     

In [ ]:
proj_summary, proj_pred_summary, laminar_profs = MesoPred.PlotStatistics(ClfResults_ridge)
#clear_output()

In [ ]:
Components = pk.load(open('saves/Components.pkl','rb'))
Code = pk.load(open('saves/Code.pkl','rb'))
dlsc_mdl = pk.load(open('saves/dict_mdl.pkl','rb'))
G_Components = pk.load(open('saves/Grange_Atoms.pkl','rb'))
G_Code = pk.load(open('saves/Grange_Code.pkl','rb'))
G_dlsc_mdl = pk.load(open('saves/Grange_mdl.pkl','rb'))

### Important Experiment: !!!!
I will remake the regionalized array by mixing projection patterns from different profiles

In [ ]:
model_meta = pk.load(open('model_meta.pkl','rb'))
layer_resolved_array = pk.load(open('layer_resolved_array.pkl','rb'))
print model_meta.keys()

In [ ]:
LayerArray,ModelMeta = MesoPred.GetLayerResolvedArray(mode = 'unionized', 
                                                      cre_file = 'Supplementary Table 1.csv')

In [ ]:
spears = sci.stats.spearmanr(new_layer_mat)
source2source = spears[0]
print source2source.shape

new_laminar_order = []; neo_layers = []; restr_source = []
for layer in np.unique(layers):
    print ' layer {}'.format(layer)
    commons = [idx for idx,val in enumerate(layers) if val == layer]
    new_laminar_order.extend(commons)
    neo_layers.extend([layers[idx] for idx in commons])
    restr_source.extend([ModelMeta['source id'][idx] for idx in commons])
    foreigners = [idx for idx,val in enumerate(layers) if val != layer]
    
new_laminar_order = np.asarray(new_laminar_order)
source2source2 = source2source[new_laminar_order,:]
source2source2 = source2source2[:,new_laminar_order]


#source_df = pd.DataFrame(data = source2source2,
#                           index = neo_layers, 
#                           columns = neo_layers) 
source_df = pd.DataFrame(data = source2source2,
                           index = restr_source, 
                           columns = restr_source) 

sil_score = metrics.silhouette_score(source2source2, neo_layers, metric = 'euclidean', random_state = 3)


In [ ]:
spears = sci.stats.spearmanr(new_layer_mat)
source2source = spears[0]
print source2source.shape

new_laminar_order = []; neo_layers = []
for layer in np.unique(layers):
    print ' layer {}'.format(layer)
    commons = [idx for idx,val in enumerate(layers) if val == layer]
    new_laminar_order.extend(commons)
    neo_layers.extend([layers[idx] for idx in commons])
    foreigners = [idx for idx,val in enumerate(layers) if val != layer]
    
new_laminar_order = np.asarray(new_laminar_order)
neo_layers        = np.asarray(neo_layers)
source2source2    = source2source[new_laminar_order,:]
source2source2    = source2source2[:,new_laminar_order]


source_df = pd.DataFrame(data = source2source2,
                           index = neo_layers, 
                           columns = neo_layers) 

sil_score = metrics.silhouette_score(source2source2, neo_layers, metric = 'euclidean', random_state = 3)
#sil_score = metrics.silhouette_score(new_layer_mat.T, layers, metric = 'euclidean', random_state = 3)
print sil_score

In [ ]:
perm_num       = 1000
prm_score_list = []
for num in range(perm_num):
    prm_ids               = np.random.permutation([idx for idx in range(len(source2source2))])
    new_layer_mat_prm     = new_layer_mat[:,prm_ids]
    new_laminar_order_prm = new_laminar_order[prm_ids]
    #neo_layers_prm        = neo_layers[prm_ids]
    
    source2source_prm     = sci.stats.spearmanr(new_layer_mat_prm)[0]
    source2source2_prm    = source2source_prm[new_laminar_order,:]
    source2source2_prm    = source2source2_prm[:,new_laminar_order]
    
    source_df = pd.DataFrame(data = source2source2_prm,
                            index = neo_layers, 
                            columns = neo_layers) 
    
    sil_score_prm = metrics.silhouette_score(source2source2_prm, neo_layers, metric = 'euclidean', random_state = 3)
    #sil_score_prm = metrics.silhouette_score(new_layer_mat_prm.T, layers[prm_ids], metric = 'euclidean', random_state = 3) 
    prm_score_list.append(sil_score_prm)
    
    
# pvallssss
laminar_pval = len([val for val in prm_score_list if val >= sil_score])/(1.0*perm_num)
print laminar_pval

In [ ]:
fp = h5py.File('for_TVMB/Merge_TVMB_pred_Connectivity.h5','r')
prim_strs =  fp['region_labels'][()]
fp.close()
struct_dict = OrderedDict([(val[val.index(' ') + 1:],structure['acronym'][idx]) for val in prim_strs for idx in range(len(structure['acronym'])) if val[val.index(' ') + 1:] == structure['name'][idx]])
struct_dict = pd.DataFrame(struct_dict, index = ['brain regions acronyms']).T
struct_dict.to_csv('paper1_figures/primary_structures.csv')

In [ ]:
plt.figure(figsize = (15,10))
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 18)
ax.yaxis.set_tick_params(labelsize = 18)
sns.heatmap(source_df)
plt.xticks(rotation = 360-90)
plt.savefig('paper1_figures/laminar_heatmap.eps')
cax = plt.gcf().axes[-1]
cax.tick_params(labelsize = 16)

In [ ]:
pk.dump(new_layer_mat, open('paper1_figures/new_layer_mat.pkl','wb'))
pk.dump(ModelMeta, open('paper1_figures/ModelMeta.pkl','wb'))
new_layer_mat = pk.load(open('paper1_figures/new_layer_mat.pkl','rb'))
ModelMeta = pk.load(open('paper1_figures/ModelMeta.pkl','rb'))
print model_meta.keys()

In [ ]:
LayerArray2 = []
for modelaki in ModelMeta['model']:
    LayerArray2.append(modelaki.connection_density.T)
     
LayerArray2 = np.dstack(LayerArray2)

In [ ]:
new_layer_mat = np.zeros((np.shape(LayerArray2)[0],np.shape(LayerArray2)[1]))
with open('structures.csv','rb') as fp:
                structure       = PL.readtable(fp) 
                struct_idx_dict = {val:idx for idx,val in enumerate(structure['id'])}
    
layers = PL.LaminarRegistration(ModelMeta['source id'])
for idx,source in enumerate(layers):
    prof_hit = [prof_id for prof_id,val in enumerate(ModelMeta['layer profiles'])                 
                if val.split(' ')[0].replace('L','layer').split('layer')[1] 
                in source.split('layer ')[1] and val != 'layer inspecific']
         
    if len(prof_hit) == 0:
        new_layer_mat[:,idx] = LayerArray2[:,idx,5]
    else:
        new_layer_mat[:,idx] = LayerArray2[:,idx,prof_hit[0]]

In [ ]:
target  = np.asarray(ModelMeta['target id'])
source  = np.asarray(ModelMeta['source id'])
resorting_trg_indices = np.argsort(ModelMeta['target id'])
#resorting_trg_indices = np.argsort(layers)
model_meta['sorting target id'] = target[resorting_trg_indices]
resorting_src_indices = np.argsort(ModelMeta['source id'])
model_meta['sorting source id'] = source[resorting_src_indices]
layer_array_2 = new_layer_mat[resorting_trg_indices,:]
layer_array_3 = layer_array_2[:,resorting_src_indices]

In [ ]:
LayerDf = pd.DataFrame(data = layer_array_3,
                          index = model_meta['sorting target id'], 
                           columns = model_meta['sorting source id'])
pk.dump(LayerDf, open('paper1_figures/Layer_specific_Df.pkl','wb'))

#LayerDf = pk.load(open('paper1_figures/Layer_specific_Df.pkl','rb'))
plt.figure(figsize = (30,15))
plt.title('laminar specific regionalized array', fontsize = 32)
sns.set(font_scale=2.1)
sns.heatmap(LayerDf)
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 22)
ax.yaxis.set_tick_params(labelsize = 22)
plt.xticks(rotation = 290)
plt.yticks(rotation = 30)
plt.savefig('paper1_figures/Merged_RegionalizedArray.jpg')
plt.savefig('paper1_figures/Merged_RegionalizedArray.eps')

In [ ]:
with open('sortedHierarchy.json','r') as fp:
     hierarchy_in_structs = json.load(fp)
        
        
targets = np.asarray([MesoPred.targetprofiles['str_acronym'][struct_idx_dict[str(val)]] for val in model_meta['target id']])
sources = np.asarray([MesoPred.targetprofiles['str_acronym'][struct_idx_dict[str(val)]] for val in model_meta['source id']])
resorting_trg_indices = np.argsort(targets)
model_meta['sorting target id'] = targets[resorting_trg_indices]
resorting_src_indices = np.argsort(sources)
model_meta['sorting source id'] = sources[resorting_src_indices]

layer_array_2 = layer_resolved_array[:,resorting_trg_indices,:]
layer_array_3 = layer_array_2[:,:,resorting_src_indices]

for idx,profile in enumerate(model_meta['layer profiles']):
     
        
    LayerDf = pd.DataFrame(data = layer_array_3[idx,0:25,0:25],
                           index = model_meta['sorting target id'][0:25], 
                           columns = model_meta['sorting source id'][0:25])
    plt.figure(figsize = (30,15))
    sns.heatmap(LayerDf)
    sns.set(font_scale = 2.3)
    plt.title(profile, fontsize = 36)
    ax = plt.gca()
    ax.xaxis.set_tick_params(labelsize = 26)
    ax.yaxis.set_tick_params(labelsize = 26)
    plt.xticks(rotation = 290)
    plt.yticks(rotation = 30)
    plt.savefig('paper1_figures/RegionalizedArray_{}.jpg'.format(profile.replace('/','')))
    plt.savefig('paper1_figures/RegionalizedArray_{}.eps'.format(profile.replace('/','')))

### Here I try a kernel regression model

In [ ]:
def myKernel(X_1,X_2):
    return np.dot(np.dot(X_1[:,0:len(X_1[0])/2],X_2[:,0:len(X_2[0])/2].T), np.dot(X_1[:,len(X_1[0])/2:len(X_1[0])],X_2[:,len(X_2[0])/2:len(X_2[0])].T).T)
    #return np.dot(x1)
    
#def myKernel(X_1,X_2):
#    return np.dot(np.dot(X_1[:,0:len(X_1[0])/2],X_2[:,0:len(X_2[0])/2].T), np.dot(X_1[:,len(X_1[0])/2:len(X_1[0])],X_2[:,len(X_2[0])/2:len(X_2[0])].T).T)    

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics.pairwise import pairwise_kernels,linear_kernel
KernelGeneMat = np.zeros((np.shape(G_red_proc)[0]**2,np.shape(G_red_proc)[1]*2))
Y_vectorized = np.reshape(Y_red_proc,(np.shape(Y_red_proc)[0]*np.shape(Y_red_proc)[1],1))
Y_vectorized = np.ravel(Y_vectorized)
outer_cv = KFold(n_splits = 6, shuffle = True, random_state = 123)
cnt = 0
for target in range(len(G_red_proc)):
    for source in range(len(G_red_proc)):
        KernelGeneMat[cnt,:] = np.concatenate((G_red_proc[target,:],G_red_proc[source,:]), axis = 0)
        cnt+=1

svr = SVR(kernel=myKernel) 
svr_pred = cross_val_predict(svr, X = KernelGeneMat, y = Y_vectorized, cv = outer_cv)
print 'svr model has been fitted'

KernelMat = np.kron(G_red_proc,G_red_proc)

#SVR(kernel=myKernel).fit(KernelMat,Y_vectorized)

### Here I do performance comparison between cre-line and wild-type data (boxplot)

In [ ]:
#all_scores_prm = np.asarray([tracer for key in ridge_pred_prm.keys()\
#                                       for tracer in ridge_pred_prm[key]])  

plt.figure(figsize = (16,6))
box1 = plt.boxplot([module_based_pred['Cux2-IRES-Cre']['r2'],module_based_pred['Emx1-IRES-Cre']['r2'],module_based_pred['wild_type']['r2']], 0, 'gD', widths = 0.6, whis = [5,95])
box2 = plt.boxplot([ridge_pred_prm['Cux2-IRES-Cre'],ridge_pred_prm['Emx1-IRES-Cre'],ridge_pred_prm['wild_type']], 0, 'r', widths = 0.6, whis = [5,95])
#box1 = plt.boxplot([ridge_pred[key] for key in ridge_pred.keys()], 0, 'gD')
#box2 = plt.boxplot([ridge_pred_prm[key] for key in ridge_pred.keys()], 0, 'r')   
for key in box1.keys():
        for bp1,bp2 in zip(box1[key],box2[key]):
            bp1.set(linewidth = 2)
            bp2.set(linewidth = 2)
plt.xticks([1,2,3],['Cux2-IRES-Cre','Emx1-IRES-Cre','wild_type'])
#plt.xticks([idx+1 for idx in range(len(ridge_pred.keys()))],[key for key in ridge_pred.keys()], rotation = 300)
plt.yticks(np.arange(0.0,1.1,0.1))
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 16)
ax.yaxis.set_tick_params(labelsize = 20)
plt.ylabel(r'$r^2$', fontsize = 20)
#plt.savefig('paper1_figures/for_paul_20_6_19/permutation_for_modules.tiff')
#plt.savefig('paper1_figures/for_paul_20_6_19/permutation_for_modules.eps')
plt.show()

plt.figure()
box1 = plt.boxplot([module_based_pred['Cux2-IRES-Cre']['r2'],module_based_pred['Emx1-IRES-Cre']['r2'],module_based_pred['wild_type']['r2']], 0, 'gD', widths = 0.6, whis = [5,95])
for key in box1.keys():
        for bp1 in box1[key]:
            bp1.set(linewidth = 2)
plt.show()            

### Statistics regrading gene modules and projection patterns

In [ ]:
MesoPred.PlotStatistics(ClfResults_ridge, structure_summary = True,
                        laminar_summary = False, tracer_category = 'wild_type',
                        selected_tracer = 46)
MesoPred.PlotStatistics(Code, structure_summary = True,
                        laminar_summary = False, tracer_category = 'gene module',
                        selected_tracer = 118)
MesoPred.PlotStatistics(Code, structure_summary = True,
                        laminar_summary = False, tracer_category = 'gene module',
                        selected_tracer = 46)
MesoPred.PlotStatistics(Code, structure_summary = True,
                        laminar_summary = False, tracer_category = 'gene module',
                        selected_tracer = 117)

In [ ]:
sel_tracer = [idx for idx,val in enumerate(MesoPred.ConDict['wild_type']['structure-abbrev']) if val == 'PAA'][0]
BP_25 = PL.BrainPlotter(resolution = 25)
BP_10 = PL.BrainPlotter(resolution = 10)
BP_25.fit(projection = ClfResults_ridge['wild_type']['y_actual'][:,sel_tracer], 
          chosen_indices = MesoPred.params['remaining_indices'])
BP_10.fit(projection = ClfResults_ridge['wild_type']['y_actual'][:,sel_tracer], 
          chosen_indices = MesoPred.params['remaining_indices'])

BP_25.plot_slice(savefile = 'paper1_figures/brain_slice_wild_type_{}.jpg'.format(sel_tracer))
BP_10.plot_flatmap(savefile = 'paper1_figures/flatmap_wild_type_{}.jpg'.format(sel_tracer))

In [ ]:
sel_tracer = [idx for idx,val in enumerate(MesoPred.ConDict['wild_type']['structure-abbrev']) if val == 'PAA'][0]
for modulo in [46,117,118]:
    rho,pval = sci.stats.pearsonr(Code[:,modulo],ClfResults_ridge['wild_type']['y_actual'][:,sel_tracer])
    print rho, modulo
    BP_25 = PL.BrainPlotter(resolution = 25)
    BP_10 = PL.BrainPlotter(resolution = 10)
    BP_25.fit(projection = Code[:,modulo], chosen_indices = MesoPred.params['remaining_indices'])
    BP_10.fit(projection = Code[:,modulo], chosen_indices = MesoPred.params['remaining_indices'])
    BP_25.plot_slice()
    BP_10.plot_flatmap()

In [ ]:
sel_tracer = [idx for idx,val in enumerate(MesoPred.ConDict['wild_type']['structure-abbrev']) if val == 'PAA'][0]
for code in range(len(G_Code[0])):
    rho = sci.stats.pearsonr(ClfResults_ridge['wild_type']['y_actual'][:,sel_tracer], G_Code[:,code])[0]
    if rho > 0.2:
        print code
        BP_25 = PL.BrainPlotter(resolution = 25)
        BP_10 = PL.BrainPlotter(resolution = 10)
        BP_25.fit(projection = G_Code[:,code], chosen_indices = MesoPred.params['remaining_indices'])
        BP_10.fit(projection = G_Code[:,code], chosen_indices = MesoPred.params['remaining_indices'])
        BP_25.plot_slice(savefile = 'paper1_figures/grange_brain_slice_module_{}.jpg'.format(code))
        BP_10.plot_flatmap(savefile = 'paper1_figures/grange_flatmap_module_{}.jpg'.format(code))
 

### Here I assess the decomposition quality of DLSC by chosing different set sizes and predicting the tract tracing data

In [ ]:
Store_now = {'scores': [], 'comp' : []}
Comp_set = [10,25,50,100,150,200,250,300,400,500,1000]
ridge_params = {'alpha': [1,10,100,250,500,1000],
                     'fit_intercept' : [True], 'normalize' : [False]}

for comp in Comp_set:
    Atoms, Code, dlsc = MesoPred.DictionaryDecomposition(ClfResults_ridge['wild_type']['Gene Expression'], 
                                                         n_comp = comp)
    ridge_pred = {}
    for key in ClfResults_ridge.keys():
        module_to_proj = []
        y_pred = np.zeros(np.shape(ClfResults_ridge[key]['y_actual']))
        Code_scale = StandardScaler().fit_transform(Code) 
        y_meas_scale = StandardScaler().fit_transform(ClfResults_ridge[key]['y_actual'])
        for train, test in MesoPred.params['validation'].split(Code,ClfResults_ridge[key]['y_actual']):
            ridge_mdl = GridSearchCV(Ridge(), param_grid = ridge_params, 
                                 scoring = 'neg_mean_squared_error', 
                                 cv = 3, n_jobs = -1)
            ridge_mdl.fit(Code_scale[train], y_meas_scale[train])
            y_pred[test] = ridge_mdl.predict(Code_scale[test]) 
        ridge_pred[key] = metrics.r2_score(y_meas_scale, y_pred, multioutput = 'raw_values')
    all_scores = np.asarray([tracer for key in ridge_pred.keys()\
                                           for tracer in ridge_pred[key]])   
    print comp, np.max(all_scores), np.min(all_scores), np.median(all_scores)
    Store_now['scores'].append(all_scores)

            
scores   = [np.median(val) for val in Store_now['scores']]
coef_num = [str(val) for val in Comp_set]
coef_num.pop(6)
pk.dump([scores,coef_num],open('paper1_figures/dictionary_to_performance.pkl','wb'))

#[scores,coef_num] = pk.load(open('paper1_figures/dictionary_to_performance.pkl','rb'))
plt.figure(figsize = (12,6))
plt.bar(coef_num, scores)
ax = plt.gca(); ax.xaxis.set_tick_params(labelsize = 16); ax.yaxis.set_tick_params(labelsize = 16)
plt.ylabel(r'median $r^2$',fontsize = 18)
plt.xlabel('dictionary set size', fontsize = 18)
plt.savefig('paper1_figures/dictionary_to_performance.jpg')
plt.savefig('paper1_figures/dictionary_to_performance.eps')
print scores
print coef_num        

### Here I make a prediction-derived scatter plot where I colorcode subcortical and cortical projections

In [ ]:
for key in ['Cux2-IRES-Cre']:
    top_tracer = np.argsort(ClfResults_ridge[key]['r2'])[1][0]
    print top_tracer, ClfResults_ridge[key]['r2'][top_tracer], MesoPred.ConDict[key]['structure-abbrev'][top_tracer]
    
A = ClfResults_ridge[key]['y_actual'][:,top_tracer]
B = ClfResults_ridge[key]['y_preds'][:,top_tracer]
laminar_profs = [MesoPred.targetprofiles['alt laminar profiles'][val] for val in MesoPred.params['remaining_indices']]

resids              = abs(A-B)

remaining_indices   = np.asarray([MesoPred.targetprofiles['str_acronym'][val] for val in MesoPred.params['remaining_indices']])
subcortical_indices = [idx for idx,val in enumerate(laminar_profs) 
                       if val == 'layer inspecific' or val == 'other profiles']
cortical_indices    = [idx for idx,val in enumerate(laminar_profs) 
                       if val != 'layer inspecific' and val != 'other profiles']

subcortical_bad     = remaining_indices[subcortical_indices][np.argsort(resids[subcortical_indices])[::-1][0:50]]
subcortical_good    = remaining_indices[subcortical_indices][np.argsort(resids[subcortical_indices])[0:50]]
cortical_bad        = remaining_indices[cortical_indices][np.argsort(resids[cortical_indices])[::-1][0:10]]
cortical_good       = remaining_indices[cortical_indices][np.argsort(resids[cortical_indices])[0:10]]

from scipy.stats import zscore
A = zscore(A); A = np.array(A,np.float16)
B = zscore(B); B = np.array(B,np.float16)

v_min = min(np.min(A),np.min(B))
v_max = max(np.max(A),np.max(B))

plt.figure(figsize = (15,12))
#plt.xticks(np.arange(-0.1,1.05,0.1))
#plt.yticks(np.arange(-0.1,1.05,0.1))
plt.xticks(np.arange(np.round(v_min)-0.2,np.round(v_max)+0.2,0.2))
plt.yticks(np.arange(np.round(v_min)-0.2,np.round(v_max)+0.2,0.2))

plt.scatter(A[subcortical_indices],B[subcortical_indices], c = 'g')
plt.legend()
plt.scatter(A[cortical_indices],B[cortical_indices], c = 'r')
plt.legend(['subcortical projections','cortical projections'],loc="lower right", prop={'size': 28})
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 22, rotation = 45) #290
ax.yaxis.set_tick_params(labelsize = 22)
#plt.axis('scaled')
plt.xlabel('Normalized projection volume (measured)', fontsize = 28)
plt.ylabel('Normalized projection volume (predicted)', fontsize = 28)
plt.title('Cux2-IRES-Cre AId', fontsize = 28)

plt.plot((v_min,v_max),(v_min,v_max))
#plt.plot((0,1),(0,1))
plt.savefig('paper1_figures/scatter_plot_for_aid.tiff',  bbox_inches = 'tight')
plt.savefig('paper1_figures/scatter_plot_for_aid.eps',  bbox_inches = 'tight')
plt.show()

### r2 and AURoc Validation for all gene expression-based predictions

In [ ]:
team_ridge = [list(pred) for key in MesoPred.ConDict.keys() for pred in ClfResults_ridge[key]['r2']]
team_auc = [list(pred) for key in MesoPred.ConDict.keys() for pred in ClfResults_ridge[key]['AURoc']]

plt.scatter(team_ridge, team_auc)
plt.plot((0,1),(0,1))
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 32)
ax.yaxis.set_tick_params(labelsize = 32)
plt.xlabel('AURoC values', fontsize = 34)
plt.ylabel(r'$r^2$ values', fontsize = 34)
plt.title('Scatter Plot', fontsize = 34)
plt.savefig('paper1_figures/r2_to_auc.jpg')
plt.savefig('paper1_figures/r2_to_auc.eps')
plt.show()

In [ ]:
plt.figure()
plt.boxplot(team_auc)
ax = plt.gca()
ax.yaxis.set_tick_params(labelsize = 20)
plt.ylabel('AURoc values', fontsize = 30)
plt.xlabel('Ridge Regression', fontsize = 30)
ax.yaxis.set_tick_params(labelsize = 20)
plt.yticks(np.arange(0.7,1.0,0.05))
plt.savefig('Total_AUC.eps')
plt.show()

total_performance = OrderedDict()
for key in ClfResults_ridge.keys():
    total_performance[key] = np.median(np.ndarray.flatten(np.asarray([val for val in ClfResults_ridge[key]['AURoc']])))

import operator
srt_total_performance = sorted(total_performance.items(), key=operator.itemgetter(1))[::-1] 
pk.dump(srt_total_performance, open('auc_srt_total_performance.pkl','wb')) 
#srt_total_performance = pk.load(open('auc_srt_total_performance.pkl','rb'))

tmp = [val[1] for val in srt_total_performance]
fig = plt.figure(figsize = (35,12))
ax  = plt.gca()
bara = plt.bar([val[0] for val in srt_total_performance], tmp, bottom = 0, align = 'center', 
        linewidth = 0.4, width = 0.6) 
bara.patches[3].set_facecolor('red')
ax.plot([-0.5, 14.5], [np.min(tmp), np.min(tmp)], "k--")
plt.xticks(rotation = -20)
plt.yticks(np.arange(0,1.1,0.1))
ax.xaxis.set_tick_params(labelsize = 24)
ax.yaxis.set_tick_params(labelsize = 36)
#plt.tight_layout()
plt.title('Median predictive performance per cre-line', fontsize = 36)
plt.ylabel('area under the ROC curve', fontsize = 36)
plt.savefig('paper1_figures/total_auc.jpg')
plt.savefig('paper1_figures/total_auc.eps')
plt.show()

In [ ]:
plt.boxplot(team_ridge)
ax = plt.gca()
ax.yaxis.set_tick_params(labelsize = 20)
plt.ylabel(r'$r^2$ values', fontsize = 30)
plt.xlabel('Ridge Regression', fontsize = 30)
ax.yaxis.set_tick_params(labelsize = 20)
plt.yticks(np.arange(0.7,1.0,0.05))
plt.savefig('Total_r2.eps')
plt.show()

In [ ]:
cre_res = [r2 for key,value in ClfResults_ridge.items() for r2 in value['r2'] if key != 'wild_type']
input_data = [cre_res,ClfResults_ridge['wild_type']['r2']]
intitle = 'Performance comparison between cre-line and wild-type data'
Custom_Boxplot(input_data, savename = 'paper1_figures/r2_cre_vs_wt', title = intitle, 
               ylabel = r'$r^2$', xticklabel = ['cre-line','wild_type'],
               inrange = [0,1.05,0.1])

### Here I run a statistical test to measure the significance of variability in performance between cre-line and wild-type data

In [ ]:
sample_num = 1000
srt_total_performance = pk.load(open('r2_srt_total_performance.pkl','rb'))
total_cre_r2 = [tmp[1] for tmp in srt_total_performance if tmp[0] != 'wild_type']
sample_size_sets = [len(ClfResults_ridge[key]['y_actual'][0]) for key in ClfResults_ridge.keys() if key != 'wild_type']
cre_median = np.median(total_cre_r2); cre_iqr = sci.stats.iqr(total_cre_r2)
wt_size = np.shape(ClfResults_ridge['wild_type']['y_actual'])[1]

cnt1 = 0; cnt2 = 0; med_dev = []; iqr_dev = []
#for sample_set in sample_size_sets:
for key in ClfResults_ridge.keys(): 
    if key == 'wild_type': continue
    sample_set = np.shape(ClfResults_ridge[key]['y_actual'])[1]
    for sample_run in range(sample_num):
        wt_subset = np.random.choice(wt_size, sample_set)
        A = ClfResults_ridge['wild_type']['y_actual'][:,wt_subset]
        B = ClfResults_ridge['wild_type']['y_preds'][:,wt_subset]
        wt_r2_sub = metrics.r2_score(A,B)
        #if np.max(wt_r2_sub) > np.max(total_cre_r2) or np.min(wt_r2_sub) < np.min(total_cre_r2):
        #if wt_r2_sub > np.max(total_cre_r2) or wt_r2_sub < np.min(total_cre_r2): 
        #print wt_r2_sub, cre_median + 1*cre_iqr, cre_median - 1*cre_iqr
        if wt_r2_sub > cre_median + 1*cre_iqr or wt_r2_sub < cre_median - 1*cre_iqr:  
            cnt1 += 1
            
print cnt1/(1.0*len(sample_size_sets)*sample_num), len(sample_size_sets)*sample_num

### Permutation Testing of CCP (Gene Expression)

In [2]:
ridge_params = {'alpha': [10.0,100.0,1000.0,10000.0, 100000.0, 10e6,10e12],'fit_intercept': [True],
                'normalize': [False]}

for key in ClfResults_ridge.keys():
    print key
    ConStr = ConDict[key]['ConMat']
    GeneExp_proc, sc_gene_exp, ConStr_proc, sc_sqrt = MesoPred.PreProcessing(
                                                                GeneExp[MesoPred.params['leaf_keys'],:], 
                                                                MesoPred.params, 
                                                                ConStr[MesoPred.params['leaf_keys'],:])
    
    y_pred = np.zeros(np.shape(ConStr_proc))
    for prm_num in range(25):
        ConStr_sqrt_ctr_prm = np.random.permutation(ConStr_proc)
        cnt = 0
        for train, test in MesoPred.params['validation'].split(GeneExp_proc,ConStr_sqrt_ctr_prm):
            cnt += 1
            surrogate_mdl = GridSearchCV(Ridge(), param_grid = ridge_params, 
                                         scoring = 'neg_mean_squared_error', 
                                         cv = 3, n_jobs = -1)
            surrogate_mdl.fit(GeneExp_proc[train], ConStr_sqrt_ctr_prm[train])
            y_pred[test] = surrogate_mdl.predict(GeneExp_proc[test])

        ridge_pred = metrics.r2_score(ConStr_sqrt_ctr_prm, y_pred, multioutput = 'raw_values')
        print np.mean(ridge_pred)
        plt.scatter(np.ndarray.flatten(ConStr_sqrt_ctr_prm),np.ndarray.flatten(y_pred))
        plt.show()
        #ridge_pred  = CustomCrossval(params['primary models'][6], MesoPred.GeneExp, ConStr_sqrt_ctr_prm, params['validation'][1])
        #ClfResults_ridge = MesoPred.UnravelResults(ridge_pred, params, sc_sqrt)

        pk.dump([y_pred, ConStr_sqrt_ctr_prm], open('scrambled_debug/' + 'regression'+ '_'+ MesoPred.params['cre-line'] +'_ridge_pred' + str(prm_num) + '.pkl','wb'))
        pk.dump(ridge_pred, open('scrambled_debug/' + key + '_all_results' + str(prm_num) + '_ridge.pkl', 'wb'))
        print 'Ridge regression num ' + str(prm_num) + ' has been completed'

NameError: name 'ClfResults_ridge' is not defined

### Ridge Regression outperforms Random Forest with a median r2 score over all 1397 tracers of 0.54 instead of 0.42

In [ ]:
team_rf    = [list(pred) for key in MesoPred.ConDict.keys() for pred in ClfResults['random forest'][key]['r2']]
team_ridge = [list(pred) for key in MesoPred.ConDict.keys() for pred in ClfResults_ridge['ridge'][key]['r2']]
print np.median(team_rf),np.median(team_ridge), sci.stats.iqr(team_rf), sci.stats.iqr(team_ridge)
JoinResults = np.asarray([(a[0],b[0]) for a,b in zip(team_ridge, team_rf)])
pk.dump(JoinResults, open('paper1_figures/rf_ridge_Results.pkl','wb'))
#JoinResults = pk.load(open('paper1_figures/rf_ridge_Results.pkl','rb'))

fig = plt.figure(figsize = (35,15))
ax = plt.gca()
plt.rcParams['axes.facecolor'] = 'white'
box = plt.boxplot(JoinResults,0, 'gD', widths = 0.6, whis = [5,95])

#plt.tight_layout()
plt.rcParams['figure.figsize']
plt.ylabel(r'$r^2$', fontsize = 36)
plt.xticks([i+1 for i in range(2)], ['Ridge Regression','Random Forest'],rotation = 0)
ax.xaxis.set_tick_params(labelsize = 36)
ax.yaxis.set_tick_params(labelsize = 36)
plt.yticks(np.arange(0,1.1,0.1))
for key in box.keys():
    for bp in box[key]:
        bp.set(linewidth = 2.5)
plt.savefig('paper1_figures/Ridge Regression - Random Forest comparison.eps')
plt.show()
plt.clf()

#JoinResults = pk.load(open('paper1_figures/rf_ridge_Results.pkl','rb'))
Custom_Boxplot(JoinResults, savename = 'paper1_figures/Ridge Regression - Random Forest comparison', 
               ylabel = r'$r^2$', xticklabel = ['Ridge Regression','Random Forest'],
               inrange = [0,1.05,0.1])

### Estimating the NaN frequency of the unionized gene expression and normalized projection volume data

In [ ]:
B = np.asarray(MesoPred.ConDict['wild_type']['ConMat'])
pk.dump(B, open('paper1_figures/ConMat_for_hist.pkl','wb'))

#B = pk.load(open('/home/ntimonidis/knoppers/Nestor/venv/paper1_figures/ConMat_for_hist.pkl','rb'))
B = B[MesoPred.params['leaf_keys'],:]
nan_freq = [len(con_area[np.isnan(con_area)])/(1.0*len(con_area)) for idx, con_area in enumerate(B)]
group_1 = len([val for val in nan_freq if val < 0.1])
group_2 = len([val for val in nan_freq if val > 0.8])
plt.figure(figsize = (12,6))
#plt.hist(np.asarray(nan_freq))
group_labels = ['less than 10%','more than 80%']
plt.bar(group_labels,[group_1,group_2])
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 18)
ax.yaxis.set_tick_params(labelsize = 18)
plt.ylabel('frequency of brain regions', fontsize = 20)
plt.xlabel('frequency of nan values per region', fontsize = 20)
plt.savefig('paper1_figures/nan_histogram2.eps')

In [ ]:
A = np.asarray(GeneExp)
nan_freq = [len(gene_area[np.isnan(gene_area)])/(1.0*len(gene_area)) for idx, gene_area in enumerate(A)]
plt.figure(figsize = (12,6))
plt.hist(np.asarray(nan_freq))
plt.ylabel('frequency of brain regions', fontsize = 12)
plt.xlabel('frequency of nan values per region', fontsize = 12)
plt.savefig('paper1_figures/nan_histogram.eps')

In [ ]:
import seaborn as sns
df = pd.DataFrame(data = np.asarray(GeneExp))
pk.dump(df, open('paper1_figures/nan_gene_mat.pkl','wb'))
#df = pk.load(open('paper1_figures/nan_gene_mat.pkl','rb'))

sns.heatmap(df, yticklabels = [], xticklabels = [])
sns.set(rc={'figure.figsize':(30,15.27)},font_scale = 2.3)
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 20)
ax.yaxis.set_tick_params(labelsize = 20)
plt.xticks(rotation = 290)
plt.xlabel('Genes', fontsize = 40)
plt.ylabel('Brain Areas', fontsize = 40)
#plt.savefig('paper1_figures/nan_gene_heatmap.jpg')
plt.savefig('paper1_figures/nan_gene_heatmap.eps')
#plt.savefig('paper1_figures/nan_gene_heatmap.tiff')

In [ ]:
ConStr = MesoPred.ConDict['wild_type']['ConMat']
ConStr_im = ConStr[MesoPred.params['leaf_keys'],:]; GeneExp_im = GeneExp[MesoPred.params['leaf_keys'],:]
ConStr_im,nanCons = PL.RemoveNanStructs(0,0.1).fit(ConStr_im)
GeneExp_im = np.delete(GeneExp_im,nanCons,0)
GeneExp_im,nanGenes = PL.RemoveNanStructs(0,0.03).fit(GeneExp_im)
ConStr_im = np.delete(ConStr_im,nanGenes,0)

In [ ]:
#reduced_acros = [MesoPred.targetprofiles['str_acronym'][val] for val in MesoPred.params['remaining_indices']]
import seaborn as sns
df = pd.DataFrame(data = np.asarray(GeneExp_im), index = reduced_acros, columns = MesoPred.params['Gene Acronyms'])
pk.dump(df, open('paper1_figures/imputed_gene_mat.pkl','wb'))
#df = pk.load(open('paper1_figures/imputed_gene_mat.pkl','rb'))

sns.set(rc={'figure.figsize':(30,15.27)},font_scale = 2.3)
sns.heatmap(df, yticklabels = [], xticklabels = [])
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 20)
ax.yaxis.set_tick_params(labelsize = 20)
plt.xticks(rotation = 290)
plt.xlabel('Genes', fontsize = 40)
plt.ylabel('Brain Areas', fontsize = 40)
#plt.savefig('paper1_figures/nan_gene_imputation_1.jpg')
plt.savefig('paper1_figures/nan_gene_imputation_1.eps')
#plt.savefig('paper1_figures/nan_gene_imputation_1.tiff')

In [ ]:
cnt = 0
temp_mat  = []
acros     = []
genes     = []
sel_genes = []
sel_acros = []

red_strs = [MesoPred.params['acronyms'][val] for val in MesoPred.params['remaining_indices']]
anc = FL.GetAncestor(red_strs,mode = 'acro')

for row,area in enumerate(GeneExp_im):
    for col, gene in enumerate(area):
        if mt.isnan(gene) == True:
            if col not in sel_genes and MesoPred.params['Gene Acronyms'][col] not in sel_genes:
                cnt = cnt + 1
                genes.append(col)
                sel_genes.append(MesoPred.params['Gene Acronyms'][col])
            if row not in acros and red_strs[row] not in sel_acros and anc.values()[row] == 'CTX': 
                cnt = cnt + 1
                acros.append(row)
                sel_acros.append(red_strs[row])
        if cnt > 10:
            break

cnt = 0            
for element in range(len(red_strs)): 
    if anc.values()[element] == 'CTX':
        cnt+=1
        if red_strs[element] not in sel_acros:
            sel_acros.append(red_strs[element])
            acros.append(element)
        if MesoPred.params['Gene Acronyms'][element] not in sel_genes:
            genes.append(element)
            sel_genes.append(MesoPred.params['Gene Acronyms'][element]) 
        if cnt == 10:
            break

acros = np.asarray(acros)
genes = np.asarray(genes)  
sel_genes = np.asarray(sel_genes); sel_acros = np.asarray(sel_acros)
gene_idx = np.argsort(sel_genes); target_idx = np.argsort(sel_acros)
sel_genes = sel_genes[gene_idx]; sel_acros = sel_acros[target_idx]
acros = acros[target_idx]; genes = genes[gene_idx]
print genes,sel_genes
print acros, sel_acros

tracer2 = []; sel_tracer = []
cnt = 0
for row,area in enumerate(acros):
    for inj in range(len(ConStr_im)):
        if mt.isnan(ConStr_im[area,inj]) == True:
            if col not in tracer2 and MesoPred.ConDict['wild_type']['structure-abbrev'][inj] not in sel_tracer:
                cnt += 1
                tracer2.append(inj)
                sel_tracer.append(MesoPred.ConDict['wild_type']['structure-abbrev'][inj])
        else:
            if cnt > 0 and MesoPred.ConDict['wild_type']['structure-abbrev'][inj] not in sel_tracer:
                sel_tracer.append(MesoPred.ConDict['wild_type']['structure-abbrev'][inj])
                tracer2.append(inj)
                cnt+=1
        if cnt > 10:
            break 

A = GeneExp_im[acros,:]
B = A[:,genes]
GeneExp_im2 = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0, verbose=0, copy=True).fit_transform(GeneExp_im)
A2 = GeneExp_im2[acros,:]
B2 = A2[:,genes]

A1 = ConStr_im[acros,:]
B1 = A1[:,tracer2] 
A3 = ConStr_im[acros,:]
B1 = A1[:,tracer2]

B3 = np.copy(B1)
for task in range(len(B1[0])):
    nans = [idx for idx,val in enumerate(B1[:,task]) if mt.isnan(val) == True]
    if len(nans) > 0:
        B3[:,task] = PL.BimodalImputation().fit(B3[:,task])
        
clear_output() 

In [ ]:
df_im = pd.DataFrame(data = np.power(B, 1./3), index = sel_acros, columns = sel_genes)
pk.dump(df_im, open('paper1_figures/nan_gene_submat.pkl','wb'))
df_im2 = pd.DataFrame(data = np.power(B2, 1./3), index = sel_acros,columns = sel_genes)
pk.dump(df_im2, open('paper1_figures/imputed_gene_submat.pkl','wb'))
df_im3 = pd.DataFrame(data = np.power(B1, 1./3), index = sel_acros, columns = sel_tracer)
df_im4 = pd.DataFrame(data = np.power(B3, 1./3), index = sel_acros, columns = sel_tracer)
df_im5 = pd.DataFrame(data = StandardScaler().fit_transform(np.power(B2, 1./3)), 
                      index = sel_acros, columns = sel_genes)
df_im6 = pd.DataFrame(data = StandardScaler().fit_transform(np.power(B3, 1./3)), 
                      index = sel_acros, columns = sel_tracer)

#df_im = pk.load(open('paper1_figures/nan_gene_submat.pkl','rb'))
#df_im2 = pk.load(open('paper1_figures/imputed_gene_submat.pkl','rb'))
max_val = max(df_im.max().max(), df_im2.max().max())

plt.figure()
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 26)
ax.yaxis.set_tick_params(labelsize = 26)
sns.set(rc={'figure.figsize':(30,15.27)},font_scale = 2.2)
sns.heatmap(df_im, vmin = 0, vmax = max_val)
plt.xticks(rotation = 290)
plt.xlabel('Genes', fontsize = 32)
plt.ylabel('Target Areas', fontsize = 32)
plt.savefig('paper1_figures/nan_gene_imputed_step1.tiff', bbox_inches = 'tight')
plt.savefig('paper1_figures/nan_gene_imputed_step1.eps', bbox_inches = 'tight')
plt.show()


plt.figure()
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 26)
ax.yaxis.set_tick_params(labelsize = 26)
sns.set(rc={'figure.figsize':(30,15.27)}, font_scale = 2.2)
sns.heatmap(df_im2, vmin = 0, vmax = max_val)
plt.xticks(rotation = 290)
plt.xlabel('Genes', fontsize = 32)
plt.ylabel('Target Areas', fontsize = 32)
plt.savefig('paper1_figures/nan_gene_imputed_step2.tiff', bbox_inches = 'tight')
plt.savefig('paper1_figures/nan_gene_imputed_step2.eps', bbox_inches = 'tight')
plt.show()

max_val = max(df_im3.max().max(), df_im4.max().max())

plt.figure()
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 26)
ax.yaxis.set_tick_params(labelsize = 26)
sns.set(rc={'figure.figsize':(30,15.27)}, font_scale=2.2)
sns.heatmap(df_im3, vmin=0, vmax = max_val)
plt.xticks(rotation = 290)
plt.xlabel('Source Areas', fontsize = 32)
plt.ylabel('Target Areas', fontsize = 32)
plt.savefig('paper1_figures/nan_con_imputed_step1.tiff', bbox_inches = 'tight')
plt.savefig('paper1_figures/nan_con_imputed_step1.eps', bbox_inches = 'tight')
plt.show()

#df_im2 = pd.DataFrame(data = np.power(B2, 1./3), index = sel_acros2, columns = sel_tracer)
#pk.dump(df_im2, open('paper1_figures/imp_con_submat.pkl','wb'))
#df_im2 = pk.load(open('paper1_figures/imp_con_submat.pkl','rb'))

plt.figure()
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 26)
ax.yaxis.set_tick_params(labelsize = 26)
sns.set(rc={'figure.figsize':(30,15.27)}, font_scale = 2.3)
sns.heatmap(df_im4, vmin=0, vmax = max_val)
plt.xticks(rotation = 290)
plt.xlabel('Source Areas', fontsize = 32)
plt.ylabel('Target Areas', fontsize = 32)
plt.savefig('paper1_figures/nan_con_imputed_step2.tiff', bbox_inches = 'tight')
plt.savefig('paper1_figures/nan_con_imputed_step2.eps', bbox_inches = 'tight')
plt.show()

#pk.dump(df_im, open('paper1_figures/zscore_gene_submat.pkl','wb'))
#df_im = pk.load(open('paper1_figures/zscore_gene_submat.pkl','rb'))
v_min = min(df_im5.min().min(),df_im6.min().min())
v_max = min(df_im5.max().max(),df_im6.max().max())
plt.figure()
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 26)
ax.yaxis.set_tick_params(labelsize = 26)
sns.set(rc={'figure.figsize':(30,15.27)}, font_scale = 2.2)
sns.heatmap(df_im5, vmin = v_min, vmax = v_max)
plt.xticks(rotation = 290)
plt.xlabel('Genes', fontsize = 32)
plt.ylabel('Target Areas', fontsize = 32)
plt.savefig('paper1_figures/gene_exp_zscored.tiff', bbox_inches = 'tight')
plt.savefig('paper1_figures/gene_exp_zscored.eps', bbox_inches = 'tight')
plt.show()

#df_im2 = pk.load(open('paper1_figures/zscore_con_submat.pkl','rb'))
plt.figure()
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 26)
ax.yaxis.set_tick_params(labelsize = 26)
sns.set(rc={'figure.figsize':(30,15.27)},font_scale = 2.3)
sns.heatmap(df_im6, vmin = v_min, vmax = v_max)
plt.xticks(rotation = 290)
plt.xlabel('Source Areas', fontsize = 32)
plt.ylabel('Target Areas', fontsize = 32)
plt.savefig('paper1_figures/con_zscored.tiff', bbox_inches = 'tight')
plt.savefig('paper1_figures/con_zscored.eps', bbox_inches = 'tight')
plt.show()

In [ ]:
ConStr_im2 = B3
pk.dump([ConStr_im2,ConStr_sqrt,GeneExp_im2,GeneExp_cube], open('paper1_figures/skewness_with_transformation.pkl','wb'))

#[ConStr_im2,ConStr_sqrt,GeneExp_im2,GeneExp_cube] = pk.load(open('/home/ntimonidis/knoppers/Nestor/venv/paper1_figures/skewness_with_transformation.pkl','rb'))
plt.figure(figsize = (12,6))
rwidth = 1.0
bins = 20
fig, axs = plt.subplots(2, 2, sharey = False, sharex = False, figsize=(44,24))
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 32)
ax.yaxis.set_tick_params(labelsize = 32)
plt.setp(axs, xticks = np.arange(-4,22,1))
axs[0,0].hist(sci.stats.skew(ConStr_im2), bins = bins, rwidth = rwidth, range = (-3,21), align = 'mid', histtype = 'bar', ec='black')
axs[0,0].set_xticks(np.arange(-3,21,1))
axs[0,0].set_ylabel('number of tracing experiments', fontsize = 36)
axs[0,0].tick_params(axis = "y", labelsize = 34)
axs[0,0].tick_params(axis = "x", labelsize = 32)
#axs[0,1].title('Skewness distribution before transformation - Wild-type tracers')
#plt.figure(figsize = (12,6))
axs[0,1].hist(sci.stats.skew(ConStr_sqrt), bins = bins, rwidth = rwidth, range = (-3,21), align = 'mid', histtype = 'bar', ec='black')
axs[0,1].set_xticks(np.arange(-3,21,1))
axs[0,1].tick_params(axis = "y", labelsize = 34)
axs[0,1].tick_params(axis = "x", labelsize = 32)
axs[1,0].hist(sci.stats.skew(GeneExp_im2), bins = bins, rwidth = rwidth, range = (-3,21), align = 'mid', histtype = 'bar', ec='black')
axs[1,0].set_xticks(np.arange(-3,21,1))
axs[1,0].set_ylabel('number of genes', fontsize = 36)
axs[1,0].set_xlabel('skewness values', fontsize = 36)
axs[1,0].tick_params(axis = "y", labelsize = 34)
axs[1,0].tick_params(axis = "x", labelsize = 32)
#plt.title('Skewness distribution before transformation - Gene Expression')
#plt.figure(figsize = (12,6))
axs[1,1].hist(sci.stats.skew(GeneExp_cube), bins = bins, rwidth = rwidth, range = (-3,21), align = 'mid', histtype = 'bar', ec='black')
axs[1,1].set_xticks(np.arange(-3,21,1))
axs[1,1].tick_params(axis = "y", labelsize = 34)
axs[1,1].set_xlabel('skewness values', fontsize = 36)
#plt.title('Skewness distribution after transformation - Gene Expression')

plt.savefig('paper1_figures/Skewness_with_transformation.tiff', bbox_inches = 'tight')
plt.savefig('paper1_figures/Skewness_with_transformation.eps', bbox_inches = 'tight')

### Subcortical and Cortical visualizations of highly predicted projection volumes and their measured counterparts

In [ ]:
tracer_list = list(ClfResults_ridge.keys())
other_scores = np.asarray([(key,tracer) for key in ClfResults_ridge.keys() for tracer,score in enumerate(ClfResults_ridge[key]['r2']) if score > 0.8])
max_r = np.asarray([(np.argmax(ClfResults_ridge[key]['r2']),np.max(ClfResults_ridge[key]['r2'])) for key in ClfResults_ridge.keys()])
tracer_category = tracer_list[np.argmax(max_r[:,1])]
tracer_num = int(max_r[np.argmax(max_r[:,1]),0])
source = MesoPred.ConDict[tracer_category]['structure-abbrev'][tracer_num]
A = ClfResults_ridge[tracer_category]['y_actual'][:,tracer_num]
B = ClfResults_ridge[tracer_category]['y_preds'][:,tracer_num]
A2 = ClfResults_ridge[tracer_category]['binary_patterns'][:,tracer_num]
B2 = ClfResults_ridge[tracer_category]['pred binary_patterns'][:,tracer_num]
print tracer_category, source 

tracer_category = other_scores[0][0] 
tracer_num = int(other_scores[0][1])
print tracer_category, MesoPred.ConDict[tracer_category]['structure-abbrev'][0]
A = ClfResults_ridge[other_scores[0][0]]['y_actual'][:,int(other_scores[0][1])]
B = ClfResults_ridge[other_scores[0][0]]['y_preds'][:,int(other_scores[0][1])]


In [ ]:
BP_25 = PL.BrainPlotter(resolution = 25)
BP_10 = PL.BrainPlotter(resolution = 10)
source = MesoPred.ConDict[tracer_category]['structure-abbrev'][tracer_num]
BP_25.fit(projection = A, chosen_indices = MesoPred.params['remaining_indices'],
          projection2 = B)
BP_10.fit(projection = A, chosen_indices = MesoPred.params['remaining_indices'],
          projection2 = B)
BP_25.plot_slice(savefile = 'figures/brain slice unionized {} {}.jpg'.format(tracer_category,source))
BP_10.plot_flatmap(savefile = 'figures/flatmap_unionized {} {}.jpg'.format(tracer_category,source))
BP_25.plot_slice(savefile = 'paper1_figures/brain_slice_unionized_{}_{}.eps'.format(tracer_category,source))
BP_10.plot_flatmap(savefile = 'paper1_figures/flatmap_unionized_{}_{}.eps'.format(tracer_category,source))

In [ ]:
A2 = ClfResults_ridge[other_scores[0][0]]['binary_patterns'][:,int(other_scores[0][1])]
B2 = ClfResults_ridge[other_scores[0][0]]['pred binary_patterns'][:,int(other_scores[0][1])]
BP_25 = PL.BrainPlotter(resolution = 25)
BP_10 = PL.BrainPlotter(resolution = 10)
BP_25.fit(projection = A2, chosen_indices = MesoPred.params['remaining_indices'])
BP_10.fit(projection = A2, chosen_indices = MesoPred.params['remaining_indices'], projection2 = B2)
#BP_25.plot_slice(savefile = 'figures/brain_slice_binarized_{}_{}.jpg'.format(tracer_category,source))#BP_10.plot_flatmap(savefile = 'figures/flatmap_binarized_{}_{}.jpg'.format(tracer_category,source), 
#                   mode = 'binary')
BP_25.plot_slice(savefile = 'paper1_figures/brain_slice binarized_{}_{}.eps'.format(tracer_category,source))
BP_10.plot_flatmap(savefile = 'paper1_figures/flatmap_binarized_{}_{}.eps'.format(tracer_category,source))

## Surrogate Enrichment Test!!

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from sklearn.linear_model import RandomizedLasso
from sklearn.pipeline import Pipeline
ro.numpy2ri.activate()

ridge_params = {'alpha': [1,10,100,250,500,1000],
                     'fit_intercept' : [True], 'normalize' : [False]}
ridge_mdl = GridSearchCV(Ridge(), param_grid = ridge_params, 
                                 scoring = 'neg_mean_squared_error', 
                                 cv = 3, n_jobs = -1)
stability_selection = RandomizedLasso(normalize = False)
scaler              = StandardScaler()
pipeline            = Pipeline([('scaler', scaler), ('stability_selection', stability_selection)])

In [ ]:
y_real        = ClfResults_ridge['Cux2-IRES-Cre']['y_actual']
AltGoDict_new = {}
AltGoDict_new['Cux2-IRES-Cre'] = {}
for tracer in range(len(ClfResults_ridge['Cux2-IRES-Cre']['y_actual'][0])):
    AltGoDict_new['Cux2-IRES-Cre'][tracer] = {}
    y_meas = ClfResults_ridge['Cux2-IRES-Cre']['y_actual'][:,tracer]
    for permutation in range(5):

        red = 0
        while red == 0:
            y_shuffle = np.random.permutation(y_meas)
            red =  np.sum(y_shuffle - y_meas)
        
        pipeline.fit(ClfResults_ridge['Cux2-IRES-Cre']['Gene Expression'], y_shuffle)
        tmp = pipeline.get_params()['stability_selection'].get_support()
        stability_hits = [MesoPred.params['Gene Ids'][i] for i, x in enumerate(tmp) if x]   
        
        ridge_mdl.fit(ClfResults_ridge['Cux2-IRES-Cre']['Gene Expression'], y_shuffle)
        top_sur_genes = np.argsort(ridge_mdl.best_estimator_.coef_)[::-1]
        ridge_hits = [MesoPred.params['Gene Ids'][i] for i, x in enumerate(top_sur_genes) if x]

        ol_positions = [id2 for id1,val1 in enumerate(stability_hits) for id2,val2 in enumerate(ridge_hits) if val1==val2]
        gene_consensus = [ridge_hits[val] for val in ol_positions if val <= len(stability_hits) and ridge_hits[val] is not None]        
        
        MGP = ro.r('''
                GOenrichment <-function(GeneList){
                    source("http://bioconductor.org/biocLite.R")
                    lib_path = "~/R/x86_64-pc-linux-gnu-library/3.2/"
                    lib_path_2 = "~/anaconda2/lib/R/library/org.Mm.eg.db"
                    #biocLite("ALL")
                    #biocLite("GOstats", lib = lib_path)
                    #biocLite("Category", lib = lib_path)
                    #biocLite("genefilter", lib = lib_path)
                    #biocLite("org.Mm.eg.db", lib = lib_path_2)
                    #biocLite("gage", lib = lib_path)
                    #biocLite(pkgs=c("Biobase", "IRanges", "AnnotationDbi"),
                     #   suppressUpdates=FALSE,
                      #  suppressAutoUpdate=FALSE,
                       # siteRepos=character(),
                        #ask=TRUE, lib= lib_path)
                    #install.packages("RSQLite")
                    #install.packages("devtools")
                    #require(devtools)
                    #library("RSQLite")
                    #library("gage")
                    #library("genefilter")
                    #library("org.Mm.eg.db")
                    library(DBI)
                    library("GO.db")
                    library("GOstats")
                    library("Category")
                    library("annotate")
                    data(ALL, package="ALL")

                    hgCutoff <- 0.05

                    params <- new("GOHyperGParams",
                           geneIds = GeneList,
                           universeGeneIds = NULL,
                           annotation = "org.Mm.eg.db",
                           ontology = "CC",
                           pvalueCutoff = hgCutoff,
                           conditional = FALSE,
                           testDirection = "over")
                    CC <- hyperGTest(params)
                    sumCC <- data.frame(summary(CC))
                    sumCC <- subset(sumCC, select=c("Term"))

                    foo <- vector(mode="list", length = 3)
                    #foo[[1]] <- sumBP
                    #foo[[2]] <- sumMF
                    foo[[3]] <- sumCC

                    return(foo)
                }''')
        r_getname = ro.globalenv['GOenrichment']
        verdict   = r_getname(gene_consensus)
        temp_list = np.asarray(verdict[2])[0]
        another_tmp = [element for element in temp_list if 'synap' in element or 'neur' in element]
        AltGoDict_new['Cux2-IRES-Cre'][tracer][permutation] = another_tmp
        pk.dump(AltGoDict_new, open('ontology_permutations/results.pkl','wb'))


In [ ]:
import rpy2.robjects as ro
AltGoDict2 = {}
AltGoDict2['Cux2-IRES-Cre'] = {}
gene_consensus = pk.load(open('paper1_figures/gene_consensus.pkl','rb'))     
for idx,gene_set in enumerate(gene_consensus['Cux2-IRES-Cre']):
    print idx
    MGP = ro.r('''
            GOenrichment <-function(GeneList){
                source("http://bioconductor.org/biocLite.R")
                lib_path = "~/R/x86_64-pc-linux-gnu-library/3.2/"
                lib_path_2 = "~/anaconda2/lib/R/library/org.Mm.eg.db"
                #biocLite("ALL")
                #biocLite("GOstats", lib = lib_path)
                #biocLite("Category", lib = lib_path)
                #biocLite("genefilter", lib = lib_path)
                #biocLite("org.Mm.eg.db", lib = lib_path_2)
                #biocLite("gage", lib = lib_path)
                #biocLite(pkgs=c("Biobase", "IRanges", "AnnotationDbi"),
                 #   suppressUpdates=FALSE,
                  #  suppressAutoUpdate=FALSE,
                   # siteRepos=character(),
                    #ask=TRUE, lib= lib_path)
                #install.packages("RSQLite")
                #install.packages("devtools")
                #require(devtools)
                #library("RSQLite")
                #library("gage")
                #library("genefilter")
                #library("org.Mm.eg.db")
                library(DBI)
                library("GO.db")
                library("GOstats")
                library("Category")
                library("annotate")
                data(ALL, package="ALL")

                hgCutoff <- 0.05

                params <- new("GOHyperGParams",
                       geneIds = GeneList,
                       universeGeneIds = NULL,
                       annotation = "org.Mm.eg.db",
                       ontology = "CC",
                       pvalueCutoff = hgCutoff,
                       conditional = FALSE,
                       testDirection = "over")
                CC <- hyperGTest(params)
                sumCC <- data.frame(summary(CC))
                sumCC <- subset(sumCC, select=c("Term"))

                foo <- vector(mode="list", length = 3)
                #foo[[1]] <- sumBP
                #foo[[2]] <- sumMF
                foo[[3]] <- sumCC

                return(foo)
            }''')
    r_getname = ro.globalenv['GOenrichment']
    verdict   = r_getname(gene_set)
    temp_list = np.asarray(verdict[2])[0]
    another_tmp = [element for element in temp_list if 'synap' in element or 'neur' in element]
    AltGoDict2['Cux2-IRES-Cre'][idx] = another_tmp
    pk.dump(AltGoDict2, open('ontology_permutations/results_actual.pkl','wb'))


In [ ]:
#AltGoDict_new = pk.load(open('ontology_permutations/results.pkl','rb'))
#AltGoDict2 = pk.load(open('ontology_permutations/results_actual.pkl','rb'))

ontology_comparison = []
pval = []
for tracer in AltGoDict_new['Cux2-IRES-Cre'].keys():
    num_hits_pert = [len(surrogate) for surrogate in AltGoDict_new['Cux2-IRES-Cre'][tracer].values()]
    num_hits_original = AltGoDict2['Cux2-IRES-Cre'][tracer]
    pval.append(len([hits for hits in num_hits_pert if hits > len(num_hits_original)])/5.0)
    #print tracer, pval
    ontology_comparison.append((len(num_hits_original), np.median(num_hits_pert)))
ontology_comparison = np.asarray(ontology_comparison)    
print len([val for val in pval if val > 0.05]), 1.0*len(ontology_comparison)
print ontology_comparison.shape

plt.figure(figsize=(12,8))
ax = plt.gca()
ax.xaxis.set_tick_params(labelsize = 18)
ax.yaxis.set_tick_params(labelsize = 18)
plt.scatter(ontology_comparison[:,0],ontology_comparison[:,1])
plt.plot((0,45),(0,45))
plt.xlabel('actual', fontsize = 18)
plt.ylabel('permutated', fontsize = 18)
plt.savefig('paper1_figures/ontology_comparison.eps')
plt.title('Comparison between actual and permutated ontology hits for Cux2-IRES-Cre', fontsize = 18)

In [ ]:
for path in os.listdir('../25 3 2019/'):
    if os.path.islink(path) is True:
        #call(['ln','-s','../25 3 2019/'+ path, path])
        os.unlink(path)